### References:

- [starter notebook from Y. Nakama](https://www.kaggle.com/yasufuminakama/inchi-resnet-lstm-with-attention-starter)
- [adapted notebook from Konrad](https://www.kaggle.com/yasufuminakama/inchi-resnet-lstm-with-attention-starter)
- [PyTorch tutorial on image captioning](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning)
- [two-layer RNN implementation](https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning/pull/79)

In [ ]:
import os
from matplotlib import pyplot as plt

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

import os
import gc
import re
import math
import time
import random
import shutil
import pickle
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import Levenshtein
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, Blur
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CFG class now includes a new parameter: `decoder_layers`. For illustration purposes, I am running a two-layer LSTM for 1 epoch on 100k images.

In [ ]:
print(timm.list_models(pretrained=True))

In [ ]:
#  n_channels_dict = {'efficientnet-b0': 1280, 'efficientnet-b1': 1280, 'efficientnet-b2': 1408,
#   'efficientnet-b3': 1536, 'efficientnet-b4': 1792, 'efficientnet-b5': 2048,
#   'efficientnet-b6': 2304, 'efficientnet-b7': 2560}

# This is not, to put it mildly, the most elegant solution ever - but I ran into some trouble 
# with checking the size of feature spaces programmatically inside the CFG definition.

class CFG:
    debug          = True
    apex           = False
    max_len        = 275
    print_freq     = 250
    num_workers    = 4
    model_name     = 'efficientnet_b2'
    enc_size       = 1408
#     model_name     = 'mobilenetv2_100'
#     enc_size       = 1280
#     model_name     = 'tnt_s_patch16_224'
#     enc_size       = 384
#     model_name     = 'vit_base_patch16_224'
#     enc_size       = 768
#     model_name     = 'resnet50'
#     enc_size       = 2048
    samp_size      = 10000
    size           = 288
#     size           = 224
    scheduler      = 'CosineAnnealingLR' 
    epochs         = 10
    T_max          = 4  
    encoder_lr     = 1e-4
    decoder_lr     = 4e-4
    min_lr         = 1e-6
    batch_size     = 32
    weight_decay   = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm  = 10
    attention_dim  = 256
    embed_dim      = 512
    decoder_dim    = 512
    decoder_layers = 2     # number of LSTM layers
    dropout        = 0.5
    seed           = 42
    n_fold         = 5
    trn_fold       = 0 
    train          = True
    train_path     = '../input/bms-molecular-translation/'
    prep_path      = '../input/preprocessed-stuff/'
    prev_model     = './prp/muh_best.pth'

# Functions

In [ ]:
class Tokenizer(object):
    
    def __init__(self):
        self.stoi = {}
        self.itos = {}

    def __len__(self):
        return len(self.stoi)
    
    def fit_on_texts(self, texts):
        vocab = set()
        for text in texts:
            vocab.update(text.split(' '))
        vocab = sorted(vocab)
        vocab.append('<sos>')
        vocab.append('<eos>')
        vocab.append('<pad>')
        for i, s in enumerate(vocab):
            self.stoi[s] = i
        self.itos = {item[1]: item[0] for item in self.stoi.items()}
        
    def text_to_sequence(self, text):
        sequence = []
        sequence.append(self.stoi['<sos>'])
        for s in text.split(' '):
            sequence.append(self.stoi[s])
        sequence.append(self.stoi['<eos>'])
        return sequence
    
    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            sequence = self.text_to_sequence(text)
            sequences.append(sequence)
        return sequences

    def sequence_to_text(self, sequence):
        return ''.join(list(map(lambda i: self.itos[i], sequence)))
    
    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = self.sequence_to_text(sequence)
            texts.append(text)
        return texts
    
    def predict_caption(self, sequence):
        caption = ''
        for i in sequence:
            if i == self.stoi['<eos>'] or i == self.stoi['<pad>']:
                break
            caption += self.itos[i]
        return caption
    
    def predict_captions(self, sequences):
        captions = []
        for sequence in sequences:
            caption = self.predict_caption(sequence)
            captions.append(caption)
        return captions

tokenizer = torch.load(CFG.prep_path + 'tokenizer2.pth')
print(f"tokenizer.stoi: {tokenizer.stoi}")

In [ ]:
def get_score(y_true, y_pred):
    scores = []
    for true, pred in zip(y_true, y_pred):
        score = Levenshtein.distance(true, pred)
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score


def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed = CFG.seed)

In [ ]:
# ====================================================
# Dataset
# ====================================================

class TrainDataset(Dataset):
    def __init__(self, df, tokenizer, transform=None):
        super().__init__()
        self.df         = df
        self.tokenizer  = tokenizer
        self.file_paths = df['file_path'].values
        self.labels     = df['InChI_text'].values
        self.transform  = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        if self.transform:
            augmented = self.transform(image = image)
            image     = augmented['image']
        label = self.labels[idx]
        label = self.tokenizer.text_to_sequence(label)
        label_length = len(label)
        label_length = torch.LongTensor([label_length])
        return image, torch.LongTensor(label), label_length
    

class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        super().__init__()
        self.df = df
        self.file_paths = df['file_path'].values
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
def bms_collate(batch):
    imgs, labels, label_lengths = [], [], []
    for data_point in batch:
        imgs.append(data_point[0])
        labels.append(data_point[1])
        label_lengths.append(data_point[2])
    labels = pad_sequence(labels, batch_first = True, padding_value = tokenizer.stoi["<pad>"])
    return torch.stack(imgs), labels, torch.stack(label_lengths).reshape(-1, 1)

In [ ]:
####### CNN ENCODER

class Encoder(nn.Module):
    def __init__(self, model_name = CFG.model_name, pretrained = False):
        super().__init__()
        self.cnn = timm.create_model(model_name, pretrained = pretrained)

    def forward(self, x):
        bs       = x.size(0)
        features = self.cnn.forward_features(x)
        
        features = features.permute(0, 2, 3, 1)

        return features

The class `DecoderWithAttention` is updated to support a multi-layer LSTM.

In [ ]:
####### RNN DECODER

# attention module
class Attention(nn.Module):
    '''
    Attention network for calculate attention value
    '''
    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        '''
        :param encoder_dim: input size of encoder network
        :param decoder_dim: input size of decoder network
        :param attention_dim: input size of attention network
        '''
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att    = nn.Linear(attention_dim, 1)            # linear layer to calculate values to be softmax-ed
        self.relu        = nn.ReLU()
        self.softmax     = nn.Softmax(dim = 1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        att1  = self.encoder_att(encoder_out)     # (batch_size, num_pixels, attention_dim)
        att2  = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        att   = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        alpha = self.softmax(att)                 # (batch_size, num_pixels)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim = 1)  # (batch_size, encoder_dim)
        return attention_weighted_encoding, alpha
    
    
# custom LSTM cell
def LSTMCell(input_size, hidden_size, **kwargs):
    m = nn.LSTMCell(input_size, hidden_size, **kwargs)
    for name, param in m.named_parameters():
        if 'weight' in name or 'bias' in name:
            param.data.uniform_(-0.1, 0.1)
    return m


# decoder
class DecoderWithAttention(nn.Module):
    '''
    Decoder network with attention network used for training
    '''

    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size, device, encoder_dim, dropout, num_layers):
        '''
        :param attention_dim: input size of attention network
        :param embed_dim: input size of embedding network
        :param decoder_dim: input size of decoder network
        :param vocab_size: total number of characters used in training
        :param encoder_dim: input size of encoder network
        :param num_layers: number of the LSTM layers
        :param dropout: dropout rate
        '''
        super(DecoderWithAttention, self).__init__()
        self.encoder_dim   = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim     = embed_dim
        self.decoder_dim   = decoder_dim
        self.vocab_size    = vocab_size
        self.dropout       = dropout
        self.num_layers    = num_layers
        self.device        = device
        self.attention     = Attention(encoder_dim, decoder_dim, attention_dim)  # attention network
        self.embedding     = nn.Embedding(vocab_size, embed_dim)                 # embedding layer
        self.dropout       = nn.Dropout(p = self.dropout)
        self.decode_step   = nn.ModuleList([LSTMCell(embed_dim + encoder_dim if layer == 0 else embed_dim, embed_dim) for layer in range(self.num_layers)]) # decoding LSTMCell        
        self.init_h        = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial hidden state of LSTMCell
        self.init_c        = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial cell state of LSTMCell
        self.f_beta        = nn.Linear(decoder_dim, encoder_dim)  # linear layer to create a sigmoid-activated gate
        self.sigmoid       = nn.Sigmoid()
        self.fc            = nn.Linear(decoder_dim, vocab_size)  # linear layer to find scores over vocabulary
        self.init_weights()                                      # initialize some layers with the uniform distribution

    def init_weights(self):
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def load_pretrained_embeddings(self, embeddings):
        self.embedding.weight = nn.Parameter(embeddings)

    def fine_tune_embeddings(self, fine_tune = True):
        for p in self.embedding.parameters():
            p.requires_grad = fine_tune

    def init_hidden_state(self, encoder_out):
        mean_encoder_out = encoder_out.mean(dim = 1)
        h = [self.init_h(mean_encoder_out) for i in range(self.num_layers)]  # (batch_size, decoder_dim)
        c = [self.init_c(mean_encoder_out) for i in range(self.num_layers)]
        return h, c

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        '''
        :param encoder_out: output of encoder network
        :param encoded_captions: transformed sequence from character to integer
        :param caption_lengths: length of transformed sequence
        '''
        batch_size       = encoder_out.size(0)
        encoder_dim      = encoder_out.size(-1)
        vocab_size       = self.vocab_size
        encoder_out      = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        num_pixels       = encoder_out.size(1)
        
        caption_lengths, sort_ind = caption_lengths.squeeze(1).sort(dim = 0, descending = True)
        encoder_out      = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]
        
        
        # embedding transformed sequence for vector
        embeddings = self.embedding(encoded_captions)  # (batch_size, max_caption_length, embed_dim)
        
        # Initialize LSTM state, initialize cell_vector and hidden_vector
        prev_h, prev_c = self.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)
        
        # set decode length by caption length - 1 because of omitting start token
        decode_lengths = (caption_lengths - 1).tolist()
        predictions    = torch.zeros(batch_size, max(decode_lengths), vocab_size, device = self.device)
        alphas         = torch.zeros(batch_size, max(decode_lengths), num_pixels, device = self.device)
        
        # predict sequence
        for t in range(max(decode_lengths)):
            batch_size_t = sum([l > t for l in decode_lengths])
            attention_weighted_encoding, alpha = self.attention(encoder_out[:batch_size_t],
                                                                prev_h[-1][:batch_size_t])
            gate = self.sigmoid(self.f_beta(prev_h[-1][:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
            attention_weighted_encoding = gate * attention_weighted_encoding

            input = torch.cat([embeddings[:batch_size_t, t, :], attention_weighted_encoding], dim=1)
            
            for i, rnn in enumerate(self.decode_step):
                # recurrent cell
                h, c = rnn(input, (prev_h[i][:batch_size_t], prev_c[i][:batch_size_t])) # cell_vector and hidden_vector

                # hidden state becomes the input to the next layer
                input = self.dropout(h)

                # save state for next time step
                prev_h[i] = h
                prev_c[i] = c
                
            preds = self.fc(self.dropout(h))  # (batch_size_t, vocab_size)
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :]      = alpha
            
        return predictions, encoded_captions, decode_lengths, alphas, sort_ind
    
    def predict(self, encoder_out, decode_lengths, tokenizer):
        
        # size variables
        batch_size  = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size  = self.vocab_size
        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        num_pixels  = encoder_out.size(1)
        
        # embed start tocken for LSTM input
        start_tockens = torch.ones(batch_size, dtype = torch.long, device = self.device) * tokenizer.stoi['<sos>']
        embeddings    = self.embedding(start_tockens)
        
        # initialize hidden state and cell state of LSTM cell
        h, c        = self.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)
        predictions = torch.zeros(batch_size, decode_lengths, vocab_size, device = self.device)
        
        # predict sequence
        end_condition = torch.zeros(batch_size, dtype=torch.long, device = self.device)
        for t in range(decode_lengths):
            awe, alpha = self.attention(encoder_out, h[-1])  # (s, encoder_dim), (s, num_pixels)
            gate       = self.sigmoid(self.f_beta(h[-1]))    # gating scalar, (s, encoder_dim)
            awe        = gate * awe
            
            input = torch.cat([embeddings, awe], dim=1)
 
            for j, rnn in enumerate(self.decode_step):
                at_h, at_c = rnn(input, (h[j], c[j]))  # (s, decoder_dim)
                input = self.dropout(at_h)
                h[j]  = at_h
                c[j]  = at_c
            
            preds = self.fc(self.dropout(h[-1]))  # (batch_size_t, vocab_size)
            predictions[:, t, :] = preds
            end_condition |= (torch.argmax(preds, -1) == tokenizer.stoi["<eos>"])
            if end_condition.sum() == batch_size:
                break
            embeddings = self.embedding(torch.argmax(preds, -1))
        
        return predictions
    
    # beam search
    def forward_step(self, prev_tokens, hidden, encoder_out, function):
        
        h, c = hidden
        #h, c = h.squeeze(0), c.squeeze(0)
        h, c = [hi.squeeze(0) for hi in h], [ci.squeeze(0) for ci in c]
        
        embeddings = self.embedding(prev_tokens)
        if embeddings.dim() == 3:
            embeddings = embeddings.squeeze(1)
            
        awe, alpha = self.attention(encoder_out, h[-1])  # (s, encoder_dim), (s, num_pixels)
        gate       = self.sigmoid(self.f_beta(h[-1]))    # gating scalar, (s, encoder_dim)
        awe        = gate * awe
        
        input = torch.cat([embeddings, awe], dim = 1)
        for j, rnn in enumerate(self.decode_step):
            at_h, at_c = rnn(input, (h[j], c[j]))  # (s, decoder_dim)
            input = self.dropout(at_h)
            h[j]  = at_h
            c[j]  = at_c

        preds = self.fc(self.dropout(h[-1]))  # (batch_size_t, vocab_size)

        #hidden = (h.unsqueeze(0), c.unsqueeze(0))
        hidden = [hi.unsqueeze(0) for hi in h], [ci.unsqueeze(0) for ci in c]
        predicted_softmax = function(preds, dim = 1)
        
        return predicted_softmax, hidden, None

In [ ]:
# Helper functions

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val   = 0
        self.avg   = 0
        self.sum   = 0
        self.count = 0

    def update(self, val, n=1):
        self.val    = val
        self.sum   += val * n
        self.count += n
        self.avg    = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s   = now - since
    es  = s / (percent)
    rs  = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, encoder, decoder, criterion, 
             encoder_optimizer, decoder_optimizer, epoch,
             encoder_scheduler, decoder_scheduler, device):
    
    batch_time = AverageMeter()
    data_time  = AverageMeter()
    losses     = AverageMeter()
    
    # switch to train mode
    encoder.train()
    decoder.train()
    
    start = end = time.time()
    global_step = 0
    
    text_preds = []
    
    for step, (images, labels, label_lengths) in enumerate(train_loader):
        
        # measure data loading time
        data_time.update(time.time() - end)
        
        images        = images.to(device)
        labels        = labels.to(device)
        label_lengths = label_lengths.to(device)
        batch_size    = images.size(0)
        
        
        features = encoder(images)

        predictions, caps_sorted, decode_lengths, alphas, sort_ind = decoder(features, labels, label_lengths)
    
        predicted_sequence = torch.argmax(predictions.detach().cpu(), -1).numpy()
        _text_preds        = tokenizer.predict_captions(predicted_sequence)
        text_preds.append(_text_preds)
        
        targets     = caps_sorted[:, 1:]
        
#         if step == 0:
#             targets0 = targets[0]
#             predictions0 = predictions[0]
#             targets0 = targets0[:decode_lengths[0]]
#             predictions0 = predictions0[:decode_lengths[0]]
#             score0 = criterion(predictions0, targets0)
#             print(targets0.shape)
#             print(predictions0.shape)
#             print(targets0)
#             print(predictions0)
#             print(predictions0[0][targets0[0]])
#             print('score 0', score0)

        
        
        predictions = pack_padded_sequence(predictions, decode_lengths, batch_first=True).data
        targets     = pack_padded_sequence(targets, decode_lengths, batch_first=True).data
        loss        = criterion(predictions, targets)

        
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
            
        if CFG.apex:
            with amp.scale_loss(loss, decoder_optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()
            
        encoder_grad_norm = torch.nn.utils.clip_grad_norm_(encoder.parameters(), CFG.max_grad_norm)
        decoder_grad_norm = torch.nn.utils.clip_grad_norm_(decoder.parameters(), CFG.max_grad_norm)
        
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            encoder_optimizer.step()
            decoder_optimizer.step()
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            global_step += 1
            
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Encoder Grad: {encoder_grad_norm:.4f}  '
                  'Decoder Grad: {decoder_grad_norm:.4f}  '
                  #'Encoder LR: {encoder_lr:.6f}  '
                  #'Decoder LR: {decoder_lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), 
                   batch_time        = batch_time,
                   data_time         = data_time, 
                   loss              = losses,
                   remain            = timeSince(start, float(step+1)/len(train_loader)),
                   encoder_grad_norm = encoder_grad_norm,
                   decoder_grad_norm = decoder_grad_norm,
                   #encoder_lr=encoder_scheduler.get_lr()[0],
                   #decoder_lr=decoder_scheduler.get_lr()[0],
                   ))
            
    text_preds = np.concatenate(text_preds)
    
    return losses.avg, text_preds, sort_ind


def valid_fn(valid_loader, encoder, decoder, tokenizer, criterion, device):
    
    batch_time = AverageMeter()
    data_time  = AverageMeter()
    losses     = AverageMeter()
    
    # switch to evaluation mode
    encoder.eval()
    decoder.eval()
    
    text_preds = []
    start = end = time.time()
    
    for step, (images, labels, label_lengths) in enumerate(valid_loader):
        
        # measure data loading time
        data_time.update(time.time() - end)
        
        images     = images.to(device)
        labels        = labels.to(device)
        label_lengths = label_lengths.to(device)
        batch_size = images.size(0)
        
        with torch.no_grad():
            features    = encoder(images)
            predictions = decoder.predict(features, CFG.max_len, tokenizer)
            
        predicted_sequence = torch.argmax(predictions.detach().cpu(), -1).numpy()
        _text_preds        = tokenizer.predict_captions(predicted_sequence)
        text_preds.append(_text_preds)
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  .format(
                   step, len(valid_loader), 
                   batch_time = batch_time,
                   data_time  = data_time,
                   remain     = timeSince(start, float(step+1)/len(valid_loader)),
                   ))
            
    
    
        
#         label_text = tokenizer.predict_captions(labels.detach().cpu().numpy())
    
    
#         predictions, caps_sorted, decode_lengths, alphas, sort_ind = decoder(features, labels, label_lengths)
#         targets     = caps_sorted[:, 1:]
#         predictions = pack_padded_sequence(predictions, decode_lengths, batch_first=True).data
#         targets     = pack_padded_sequence(targets, decode_lengths, batch_first=True).data
#         loss        = criterion(predictions, targets)



#         label_lengths, sort_ind = label_lengths.squeeze(1).sort(dim = 0, descending = True)
#         caps_sorted = labels[sort_ind]
        
        decode_lengths = (label_lengths.squeeze(1) - 1).tolist()
#         decode_lengths = label_lengths.squeeze(1).tolist()


#         print('decode_lengths', len(decode_lengths))
        
#         print('predictions', predictions.shape)
#         print('labels', labels.shape)


        targets = labels[:, 1:]
    
#         print('predicted_sequence', predicted_sequence.shape )
#         print(predicted_sequence[0])
#         print('targets', targets.shape)
#         print(targets[0])


#         loss22 = 0
#         num = 0
#         for i, leng in enumerate(decode_lengths):
            
#             targets0 = targets[i]
#             predictions0 = predictions[i]
# #             leng = decode_lengths[0]
#             targets0 = targets0[:leng]
#             predictions0 = predictions0[:leng]
#             score0 = criterion(predictions0, targets0)
#             loss22 += score0
#             num += 1
# #             print('targets0', targets0.shape)
# #             print('predictions0', predictions0.shape)
# #             print('score0', score0)
#         loss22 /= num
#         print('loss22', loss22)

#         if step == 0:
#             targets0 = targets[0]
#             predictions0 = predictions[0]
#             targets0 = targets0[:decode_lengths[0]]
#             predictions0 = predictions0[:decode_lengths[0]]
#             score0 = criterion(predictions0, targets0)
#             print(targets0.shape)
#             print(predictions0.shape)
#             print(targets0)
#             print(predictions0)
#             print(predictions0[0][targets0[0]])
#             print('score 0', score0)
        
    
        predictions = pack_padded_sequence(predictions, decode_lengths, batch_first=True, enforce_sorted=False).data
        targets     = pack_padded_sequence(targets, decode_lengths, batch_first=True, enforce_sorted=False).data
#         print('pad predictions', predictions.shape)
#         print('pad labels', targets.shape)
        loss        = criterion(predictions, targets)


#         print('loss val', loss)

        # record loss
        losses.update(loss.item(), batch_size)
    
    
    print('validation loss', losses.avg)
    text_preds = np.concatenate(text_preds)
    
    return text_preds

In [ ]:
# ====================================================
# Train loop
# ====================================================
encoder = Encoder(CFG.model_name, pretrained = True)

decoder = DecoderWithAttention(attention_dim = CFG.attention_dim, 
                               embed_dim     = CFG.embed_dim, 
                               encoder_dim   = CFG.enc_size,
                               decoder_dim   = CFG.decoder_dim,
                               num_layers    = CFG.decoder_layers,
                               vocab_size    = len(tokenizer), 
                               dropout       = CFG.dropout, 
                               device        = device)

def train_loop(folds, fold, encoder, decoder):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds  = folds.loc[trn_idx].reset_index(drop = True)
    valid_folds  = folds.loc[val_idx].reset_index(drop = True)
    valid_labels = valid_folds['InChI'].values
    train_labels = train_folds['InChI'].values

    train_dataset = TrainDataset(train_folds, tokenizer, transform = get_transforms(data = 'train'))
#     valid_dataset = TestDataset(valid_folds, transform = get_transforms(data = 'valid'))
    valid_dataset = TrainDataset(valid_folds, tokenizer, transform = get_transforms(data = 'valid'))


    
    train_loader = DataLoader(train_dataset, 
                              batch_size  = CFG.batch_size, 
                              shuffle     = True, 
                              num_workers = CFG.num_workers, 
                              pin_memory  = True,
                              drop_last   = True, 
                              collate_fn  = bms_collate)
#     valid_loader = DataLoader(valid_dataset, 
#                               batch_size  = CFG.batch_size, 
#                               shuffle     = False, 
#                               num_workers = CFG.num_workers,
#                               pin_memory  = True, 
#                               drop_last   = False)
    
    valid_loader = DataLoader(valid_dataset, 
                              batch_size  = CFG.batch_size, 
                              shuffle     = False, 
                              num_workers = CFG.num_workers, 
                              pin_memory  = True,
                              drop_last   = False, 
                              collate_fn  = bms_collate)
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, 
                                          mode     = 'min', 
                                          factor   = CFG.factor, 
                                          patience = CFG.patience, 
                                          verbose  = True, 
                                          eps      = CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, 
                                          T_max      = CFG.T_max, 
                                          eta_min    = CFG.min_lr, 
                                          last_epoch = -1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, 
                                                    T_0        = CFG.T_0, 
                                                    T_mult     = 1, 
                                                    eta_min    = CFG.min_lr, 
                                                    last_epoch = -1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================

#    states = torch.load(CFG.prev_model,  map_location=torch.device('cpu'))

#    encoder.load_state_dict(states['encoder'])
    
    encoder.to(device)
    encoder_optimizer = Adam(encoder.parameters(), 
                             lr           = CFG.encoder_lr, 
                             weight_decay = CFG.weight_decay, 
                             amsgrad      = False)
#    encoder_optimizer.load_state_dict(states['encoder_optimizer'])
    encoder_scheduler = get_scheduler(encoder_optimizer)
#    encoder_scheduler.load_state_dict(states['encoder_scheduler'])
    
    
#    decoder.load_state_dict(states['decoder'])
    decoder.to(device)
    decoder_optimizer = Adam(decoder.parameters(), 
                             lr           = CFG.decoder_lr, 
                             weight_decay = CFG.weight_decay, 
                             amsgrad      = False)
#    decoder_optimizer.load_state_dict(states['decoder_optimizer'])

    decoder_scheduler = get_scheduler(decoder_optimizer)
 #   decoder_scheduler.load_state_dict(states['decoder_scheduler'])

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.stoi["<pad>"])

    best_score = np.inf
    best_loss  = np.inf
    record_score = []
    record_score_train = []
    record_loss = []
    
    for epoch in range(CFG.epochs):
        
        start_time = time.time()
        
        # train
        avg_loss, text_preds_train, sort_ind = train_fn(train_loader, encoder, decoder, criterion, 
                            encoder_optimizer, decoder_optimizer, epoch, 
                            encoder_scheduler, decoder_scheduler, device)

        # eval
        text_preds = valid_fn(valid_loader, encoder, decoder, tokenizer, criterion, device)
        text_preds = [f"InChI=1S/{text}" for text in text_preds]
        text_preds_train = [f"InChI=1S/{text}" for text in text_preds_train]
        LOGGER.info(f"train labels: {train_labels[:5]}")
        LOGGER.info(f"train preds: {text_preds_train[:5]}")
#         LOGGER.info(f"val labels: {valid_labels[:5]}")
#         LOGGER.info(f"val preds: {text_preds[:5]}")
        
        # scoring
        score_train = get_score(train_labels, text_preds_train)
#         print('score_train', score_train)
        score = get_score(valid_labels, text_preds)
#         print('valid_labels', valid_labels)
        
        
        if isinstance(encoder_scheduler, ReduceLROnPlateau):
            encoder_scheduler.step(score)
        elif isinstance(encoder_scheduler, CosineAnnealingLR):
            encoder_scheduler.step()
        elif isinstance(encoder_scheduler, CosineAnnealingWarmRestarts):
            encoder_scheduler.step()
            
        if isinstance(decoder_scheduler, ReduceLROnPlateau):
            decoder_scheduler.step(score)
        elif isinstance(decoder_scheduler, CosineAnnealingLR):
            decoder_scheduler.step()
        elif isinstance(decoder_scheduler, CosineAnnealingWarmRestarts):
            decoder_scheduler.step()

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')
        
        record_score_train.append(score_train)
        record_score.append(score)
        record_loss.append(avg_loss)
        
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'encoder': encoder.state_dict(), 
                        'encoder_optimizer': encoder_optimizer.state_dict(), 
                        'encoder_scheduler': encoder_scheduler.state_dict(), 
                        'decoder': decoder.state_dict(), 
                        'decoder_optimizer': decoder_optimizer.state_dict(), 
                        'decoder_scheduler': decoder_scheduler.state_dict(), 
                        'text_preds': text_preds,
                       },
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best.pth')
          
    print("train scores", record_score_train)
    print("validation scores", record_score)
    print("train losses", record_loss)

In [ ]:
def get_train_file_path(image_id):

    return CFG.train_path + "train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

def get_test_file_path(image_id):

    return CFG.train_path + "test/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

In [ ]:
# transformations

def get_transforms(*, data):
    
    if data == 'train':
        return Compose([
            Resize(CFG.size, CFG.size),
            HorizontalFlip(p=0.5),                  
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),   
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])


# Data

In [ ]:
train = pd.read_pickle(CFG.prep_path + 'train2.pkl')

train['file_path'] = train['image_id'].apply(get_train_file_path)

print(f'train.shape: {train.shape}')

test = pd.read_csv('../input/bms-molecular-translation/sample_submission.csv')

test['file_path'] = test['image_id'].apply(get_test_file_path)

print(f'test.shape: {test.shape}')


if CFG.debug:
    # CFG.epochs = 1
    train = train.sample(n = CFG.samp_size, random_state = CFG.seed).reset_index(drop = True)

In [ ]:
train_dataset = TrainDataset(train, tokenizer, transform = get_transforms(data='train'))

folds = train.copy()
Fold = StratifiedKFold(n_splits = CFG.n_fold, shuffle = True, random_state = CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['InChI_length'])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)

In [ ]:
# train predictions torch.Size([32, 132, 193])
# train targets torch.Size([32, 132])
# train pad predictions torch.Size([2881, 193])
# train pad targets torch.Size([2881])

#  predictions torch.Size([32, 275, 193])
# labels torch.Size([32, 149])
# pad predictions torch.Size([2956, 193])
# pad labels torch.Size([2956])

# Training

In [ ]:
train_loop(folds, CFG.trn_fold, encoder, decoder)

In [ ]:
efficient_net_b1 = [98.8575, 73.6695, 72.2025, 75.6785, 72.8615, 73.5025, 67.227, 74.857, 65.8755, 63.5605, 62.218, 61.8435, 62.494, 62.4595, 60.855, 61.2445, 60.16, 59.3595, 58.445, 57.9975]
efficient_net_b1_loss = [2.3931803798675535, 1.628167809009552, 1.5203406267166137, 1.4713778057098388, 1.4598911304473876, 1.4517746849060058, 1.4298730211257935, 1.3751705694198608, 1.304869324684143, 1.237863293170929, 1.1833606758117676, 1.145661545753479, 1.1337855563163757, 1.132903272151947, 1.136840611934662, 1.1170298233032228, 1.099350608587265, 1.0599301619529724, 1.0109912855625152, 0.9796263043880462]

b2_train_score = [113.59625, 98.33, 96.615875, 94.968375, 94.140625, 94.623625, 93.863125, 92.865125, 91.84775, 91.042375]
efficient_net_b2 = [134.222, 74.5255, 80.9045, 76.0485, 74.751, 71.3455, 73.473, 66.329, 65.993, 64.856, 62.6475, 60.817, 61.522, 60.7185, 61.715, 61.191, 59.6985, 58.849, 59.1755, 56.8945]
efficient_net_b2_loss = [2.415083875179291, 1.6237234082221985, 1.5118725147247314, 1.4611049151420594, 1.4461445527076722, 1.4379349818229676, 1.409820526123047, 1.3576485114097596, 1.289951596736908, 1.2252001495361329, 1.1670255136489869, 1.1304647407531738, 1.1173807997703553, 1.1154722766876222, 1.1165171647071839, 1.1063506488800048, 1.0751800196170807, 1.0316696577072144, 0.9882933089733124, 0.9550682859420776]
b2_val_loss = [5.1344, 5.6729, 6.0253, 6.0020, 6.0312, 6.1491, 6.1898, 6.3044, 6.4776, 6.3926]

efficient_net_b3 = [95.667, 83.3315, 76.262, 73.2065, 73.8655, 73.544, 67.0345, 68.6645, 67.535, 61.066, 62.0325, 59.302, 59.429, 58.9515, 60.4385, 62.1805, 60.259, 56.178, 55.834, 55.59]
efficient_net_b3_loss = [2.3996424560546874, 1.626395184993744, 1.5165460896492005, 1.465637098789215, 1.4504963603019714, 1.443153549194336, 1.4172072825431823, 1.359358612060547, 1.286353558063507, 1.2174233260154723, 1.1592508525848388, 1.1197750024795532, 1.107436776638031, 1.1040277109146117, 1.1063713212013244, 1.096435488462448, 1.0655429661273956, 1.018177206516266, 0.9751036190986633, 0.9428163130283356]
efficient_net_b4 = [250.2225, 229.892, 111.1725, 133.9945, 134.934, 146.641, 114.107, 81.764, 79.7775, 79.208, 80.16, 74.4985, 75.5305, 74.5765, 75.9875, 74.524, 74.388, 70.883, 72.6915, 68.602]
efficient_net_b4_loss = [3.1840683708190918, 2.869856611251831, 2.4396746187210083, 2.160268536567688, 2.102885934829712, 2.0599588737487795, 1.903951898097992, 1.718906756401062, 1.5983499155044556, 1.5180621557235718, 1.4636279873847962, 1.4322679510116578, 1.423622896194458, 1.41957506275177, 1.4109104981422425, 1.3853939290046693, 1.350641189098358, 1.3104349522590637, 1.2739801769256591, 1.2522076315879822]
efficient_net_b5 = [83.609, 74.229, 74.75, 76.512, 74.7315, 73.3395, 72.2375, 82.597, 83.251, 76.9785, 76.961, 75.74, 74.002, 75.263, 70.516, 72.561, 77.1875, 70.863, 75.4075, 74.069]
efficient_net_b5_loss = [2.0829786279201508, 1.557403645992279, 1.453078752040863, 1.401730174779892, 1.3860347771644592, 1.3810959169864654, 1.3654353585243224, 1.329354391336441, 1.2842514560222626, 1.2379606404304504, 1.194757817029953, 1.1652341079711914, 1.1550245118141174, 1.1558650314807892, 1.162438308238983, 1.1578786492347717, 1.1389484195709227, 1.1131686375141143, 1.0777646358013153, 1.0513258594274522]
efficient_net_b6 = [130.5265, 86.1965, 88.264, 88.764, 80.958, 85.9995, 90.244, 97.253, 78.8195, 79.524, 98.389, 92.6205, 93.9555, 90.7505, 81.7735, 80.19, 71.238, 74.874, 72.5205, 72.3895]
efficient_net_b6_loss = [2.0776397485733034, 1.5548398282527924, 1.4560341963768004, 1.4082622706890107, 1.3940581126213074, 1.3898768017292022, 1.3772511410713195, 1.3386958220005036, 1.2897724032402038, 1.2428567588329316, 1.2017232131958009, 1.1723768224716187, 1.1614158926010132, 1.1626962089538575, 1.1686801233291626, 1.1650111904144287, 1.1489011342525481, 1.1179789177179336, 1.0860439949035645, 1.0597657492160797]

mobile_net = [255.6650, 78.9660, 70.1145, 77.7875, 74.6280, 71.4225, 73.6685, 70.1670, 71.3795, 74.1130]
tnt = [98.3080, 88.8265, 79.2645, 70.0020, 72.5620, 71.3095, 70.1475, 78.2650, 69.4255, 69.5965]
vit = [108.6810, 74.8460, 72.3420, 76.8190, 70.9615, 70.2555, 69.4180, 72.0445, 71.0110, 69.7120]
resnet50 = [164.9995, 100.7000, 83.7735, 70.8685, 70.8185, 70.9180, 70.0885, 73.7620, 70.0250, 71.5485]

In [ ]:
x1 = range(1, 21)
x = range(1, 11)
plt.plot(x, efficient_net_b1[:10])
plt.plot(x, mobile_net)
plt.plot(x, resnet50)
plt.plot(x, tnt)
plt.plot(x, vit)
plt.ylabel("Levenshtein Distance")
plt.xlabel("Epochs")
plt.ylim(50, 100)
plt.legend(['EfficientNet B1', 'MobileNet', 'ResNet50', 'TNT', 'ViT'])
plt.savefig("fig2.jpg", dpi=200)
plt.show()

In [ ]:
plt.figure()
plt.plot(x1, efficient_net_b1)
plt.plot(x1, efficient_net_b2)
plt.plot(x1, efficient_net_b3)
plt.plot(x1, efficient_net_b4)
plt.plot(x1, efficient_net_b5)
plt.plot(x1, efficient_net_b6)

plt.ylabel("Levenshtein Distance")
plt.xlabel("Epochs")
plt.xticks(range(2, 22, 2))
plt.ylim(50, 100)
plt.legend(['EfficientNet B1', 'EfficientNet B2', 'EfficientNet B3', 'EfficientNet B4', 'EfficientNet B5', 'EfficientNet B6'])
plt.savefig("fig4.jpg", dpi=200)
plt.show()

In [ ]:
plt.figure()
plt.plot(x1, efficient_net_b1_loss)
plt.plot(x1, efficient_net_b2_loss)
plt.plot(x1, efficient_net_b3_loss)
plt.plot(x1, efficient_net_b4_loss)
plt.plot(x1, efficient_net_b5_loss)
plt.plot(x1, efficient_net_b6_loss)

plt.ylabel("Cross Entropy Loss")
plt.xlabel("Epochs")
plt.xticks(range(2, 22, 2))
plt.ylim(0.8, 1.5)
plt.legend(['EfficientNet B1', 'EfficientNet B2', 'EfficientNet B3', 'EfficientNet B4', 'EfficientNet B5', 'EfficientNet B6'])
plt.savefig("fig6.jpg", dpi=200)
plt.show()

# Submission (do not run the cells below)

In [ ]:
# def inference(test_loader, encoder, decoder, tokenizer, device):
    
#     encoder.eval()
#     decoder.eval()
    
#     text_preds = []
#     tk0 = tqdm(test_loader, total = len(test_loader))
    
#     for images in tk0:
        
#         images = images.to(device)
        
#         with torch.no_grad():
#             features = encoder(images)
#             predictions = decoder.predict(features, CFG.max_len, tokenizer)
            
#         predicted_sequence = torch.argmax(predictions.detach().cpu(), -1).numpy()
#         _text_preds = tokenizer.predict_captions(predicted_sequence)
#         text_preds.append(_text_preds)
        
#     text_preds = np.concatenate(text_preds)
    
#     return text_preds

In [ ]:
# ====================================================
# inference
# ====================================================

# test_dataset = TestDataset(test, transform = get_transforms(data = 'valid'))
# test_loader  = DataLoader(test_dataset, batch_size = 256, shuffle = False, num_workers = CFG.num_workers)
# predictions  = inference(test_loader, encoder, decoder, tokenizer, device)

In [ ]:
# ====================================================
#  submission
# ====================================================

# test['InChI'] = [f"InChI=1S/{text}" for text in predictions]
# test[['image_id', 'InChI']].to_csv('submission.csv', index=False)

In [ ]:
# 问题
# 我算的 validation loss 应该不对，
train_loss = [2.415083875179291, 1.6237234082221985, 1.5118725147247314, 1.4611049151420594, 1.4461445527076722, 1.4379349818229676, 1.409820526123047, 1.3576485114097596, 1.289951596736908, 1.2252001495361329, 1.1670255136489869, 1.1304647407531738, 1.1173807997703553, 1.1154722766876222, 1.1165171647071839, 1.1063506488800048, 1.0751800196170807, 1.0316696577072144, 0.9882933089733124, 0.9550682859420776]
validation_loss = [5.1344, 5.6729, 6.0253, 6.0020, 6.0312, 6.1491, 6.1898, 6.3044, 6.4776, 6.3926]
validation_score = [134.222, 74.5255, 80.9045, 76.0485, 74.751, 71.3455, 73.473, 66.329, 65.993, 64.856, 62.6475, 60.817, 61.522, 60.7185, 61.715, 61.191, 59.6985, 58.849, 59.1755, 56.8945]
# validation Levenshtein distance score 随epochs 递减，说明model的training没问题  但是我算的 validation loss随epochs增加 不知道是哪里的问题
# validation loss 是在 valid_fn()里面算的:
# predictions = decoder.predict(features, CFG.max_len, tokenizer)
# decode_lengths = (label_lengths.squeeze(1) - 1).tolist()
# targets = labels[:, 1:]
# predictions = pack_padded_sequence(predictions, decode_lengths, batch_first=True, enforce_sorted=False).data
# targets     = pack_padded_sequence(targets, decode_lengths, batch_first=True, enforce_sorted=False).data
# loss        = criterion(predictions, targets)